In [1]:
import pandas as pd
import os
from owlready2 import *

import owlready2
print(owlready2.VERSION) ## Bugs when > 0.40 but works file at owlready2==0.40 

0.40


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
onto = get_ontology("pbn_t3_5.owl").load()

# Overview

In [3]:
debug = False
if debug:
    print("\n# Overview\n")
    for k in onto.classes():
        if len(k.instances()):
            print("* ","_"+str(k).split(".")[-1]+"_"," -- ",len(k.instances()), "instances.")
    print("\n# Details\n")
    for k in onto.classes():
        K = str(k).split(".")[-1]
        if K not in ["Qid","PBNThing"]:
            print("## ",K," -- ",len(k.instances()), "instances.")
            for i in k.instances()[:2]: 
                I = str(i).split(".")[-1]
                if K == "Qid":
                    if len(i.has_Qid_alias) > 0:
                        print("### ",I," -> ",i.has_Qid_alias[0])
                    else:
                        print("### ",I)
                    for p in list(i.get_properties()):
                        P = str(p).split(".")[-1]
                        OBJ = "onto."+str(I)+"."+str(P)
                        if not P == "label":
                            print("* ","pbn_t3_5."+str(I)+"."+str(P),": ",eval(OBJ))
                else:
                    print("### ",I," -> ",i.label[0])
                    for p in list(i.get_properties()):
                        P = str(p).split(".")[-1]
                        OBJ = "onto."+str(I)+"."+str(P)
                        if not P == "label":
                            print("* ","pbn_t3_5."+str(I)+"."+str(P),": ",eval(OBJ))


# Creating mardown files and excel

### Creating QIDs

In [15]:
LINK = "https://github.com/mm80843/T3.5/blob/main/docs/"
for k in onto.classes():
    K = str(k).split(".")[-1]
    if K in ["Qid"]:
        ClassName = K
        PATH = "./docs/"+K+"/"
        #PATH = '7.pages/'+K+"/"
        if not os.path.isdir(PATH):
            os.mkdir(PATH)
        print("QID:",len(k.instances()))
        for i in k.instances(): 
            I = str(i).split(".")[-1]
            MD = "[Home](https://github.com/mm80843/T3.5/blob/main/docs/index.md) >> Class: ["+K+"](https://github.com/mm80843/T3.5/tree/main/docs/"+K+"/index.md)"+" >> Individual ID:"+i.get_name()+" \n\n"
            MD += "# __"+i.get_name()+"__\n\n"
            FILENAME = PATH+i.get_name()+".md"
            A = list(i.get_properties())
            A.sort(key=lambda x: str(x))
            for p in A:
                P = str(p).split(".")[-1]
                OBJ = "onto."+str(I)+"."+str(P)
                OB = eval(OBJ)
                if not (OB == [onto.nan]):
                    if not P == "label":
                        if not p.label:
                            MD += "## Property: "+P+"\n\n"
                        else:
                            MD += "## "+str(p.label[0])+"\n\n"
                        E = eval(OBJ)[0]
                        
                        if type(E) is str:
                            MD += E + "\n\n"
                        else:

                            A = [x.get_name() for x in eval(OBJ)]
                            if "_" in str(A[0]):
                                A = [LINK+x.split("PBN__")[1].split("_")[0]+"/"+x+".md" for x in A]
                                #print(str(A[0]),A)
                            else:                                
                                A = [LINK+"/Qid/"+x+".md" for x in A]
                            #print(A,eval(OBJ))
                            if (OB[0].is_instance_of[0] == onto.Qid):
                                B = [x.get_name() for x in eval(OBJ)]
                            else:
                                B = [x.label[0] for x in eval(OBJ)]
                            for k in range(len(A)):
                                if not B[k] == "nan":
                                    MD += "* ["+B[k]+"]("+A[k]+")\n"
                            MD += "\n"


            #print(MD)
            with open(FILENAME,"w",encoding="utf-8") as f:
                f.write(MD)

QID: 3636


### Creating the rest

In [14]:
LINK = "https://github.com/mm80843/T3.5/blob/main/docs/"
CLASSES = []
NAMECLASSES = []
for k in onto.classes():
    K = str(k).split(".")[-1]
    if (K not in ["Qid","PBNThing"]) and len(k.instances()):
        ClassName = K
        print(K)
        NAMECLASSES.append(K)
        CLASS = {}
        PATH = "./docs/"+K+"/"
        #PATH = '7.pages/'+K+"/"
        if not os.path.isdir(PATH):
            os.mkdir(PATH)
        #print("## ",K," -- ",len(k.instances()), "instances.")
        for i in k.instances(): 
            try:
                I = str(i).split(".")[-1]
                CLASS[i] = {}
                CLASS[i]["ID"] = I
                MD = "[Home](https://github.com/mm80843/T3.5/blob/main/docs/index.md) >> Class: ["+K+"](https://github.com/mm80843/T3.5/tree/main/docs/"+K+"/index.md)"+" >> Individual ID:"+i.get_name()+" \n\n"
                MD += "# __"+i.label[0]+"__\n\n"
                FILENAME = PATH+i.get_name()+".md"
                A = list(i.get_properties())
                A.sort(key=lambda x: str(x))
                for p in A:
                    P = str(p).split(".")[-1]
                    OBJ = "onto."+str(I)+"."+str(P)
                    OB = eval(OBJ)
                    E = eval(OBJ)[0]
                    if not (OB == [onto.nan]):
                        if not P == "label":
                            if not p.label:
                                MD += "## Property: "+P+"\n\n"
                            else:
                                MD += "## "+str(p.label[0])+"\n\n"
                                

                            E = eval(OBJ)[0]

                                                        
                            if type(E) is str:
                                MD += E + "\n\n"
                                CLASS[i][P] = E
                            else:

                                A = [x.get_name() for x in eval(OBJ)]
                                if "_" in str(A[0]):
                                    A = [LINK+x.split("PBN__")[1].split("_")[0]+"/"+x+".md" for x in A]
                                    #print(str(A[0]),A)
                                else:                                
                                    A = [LINK+"/Qid/"+x+".md" for x in A]
                                #print(A,eval(OBJ))
                                if (OB[0].is_instance_of[0] == onto.Qid):
                                    B = [() for x in eval(OBJ)]
                                else:
                                    B = [x.label[0] for x in eval(OBJ)]
                                for k in range(len(A)):
                                    if not B[k] == "nan":
                                        MD += "* ["+B[k]+"]("+A[k]+")\n"
                                MD += "\n"
                                CLASS[i][P] = [str(x) for x in B]

                            CLASS[i]["NAME"] = E
                            CLASS[i]["PBNTYPE"] = ClassName                                
                        else:
                            E = eval(OBJ)[0]
                            CLASS[i]["NAME"] = E
                            CLASS[i]["PBNTYPE"] = ClassName
                #print(MD)
                with open(FILENAME,"w",encoding="utf-8") as f:
                    f.write(MD)
            except:
                pass
        CLASSES.append(CLASS)

Benef
BenefReturn
Article
Risk
RiskGBN
RiskMitigation
RiskHealth
RiskType
Stakeholder
Technology
RiskISO_Impact
RiskISO_Purpose
StakeholderGroup
StakeholderSubgroup
TechGroup
TechSubgroup
Enabler
Intervention
Blueprint
BP_Category
BP_Theme
BP_Intervention
Mitigation
PBNCategory
RiskISO


In [16]:
str(i.is_instance_of[0]).split(".")[-1],ClassName

('Qid', 'Qid')

# Creating the excel of the KG

In [17]:
ALLDFs = []
for  k in range(len(CLASSES)):
    D = pd.DataFrame(CLASSES[k]).T
    if len(D):
        D = D.convert_dtypes()
        C = list(D.columns)
        if len(C):
            DT = D.dtypes
            for x in range(len(DT)):
                if DT[x] == "object":
                    if C[x] not in [ "has_BenefitForPeople","has_Closest",  "has_SubjectRisk", "has_RiskOwner" , "has_RiskSource", "has_RiskHealth","has_RiskISO_Purpose","has_RiskISO_Impact","has_RiskType","has_RiskGBN"]:
                        D = D.explode(C[x])
            D = D.convert_dtypes()
            CC = [x for x in C if x not in ["ID","NAME","PBNTYPE"]]
            CC = ["PBNTYPE","ID","NAME"] + CC
            print(k,list(D.columns),list(CC))
            D = D[CC]
            ALLDFs.append(D)

0 ['ID', 'has_BenefDetails', 'NAME', 'PBNTYPE', 'has_BenefReturn', 'has_BenefSource', 'has_Beneficiary'] ['PBNTYPE', 'ID', 'NAME', 'has_BenefDetails', 'has_BenefReturn', 'has_BenefSource', 'has_Beneficiary']
1 ['ID', 'NAME', 'PBNTYPE'] ['PBNTYPE', 'ID', 'NAME']
2 ['ID', 'has_ArticleDOI', 'NAME', 'PBNTYPE', 'has_ArticleLLMShortSummary', 'has_ArticleLLMSummary', 'has_ArticleRef', 'has_ArticleTitle', 'has_ArticleYear', 'has_BenefInArticle', 'has_RiskInArticle', 'has_SourceMitigation'] ['PBNTYPE', 'ID', 'NAME', 'has_ArticleDOI', 'has_ArticleLLMShortSummary', 'has_ArticleLLMSummary', 'has_ArticleRef', 'has_ArticleTitle', 'has_ArticleYear', 'has_BenefInArticle', 'has_RiskInArticle', 'has_SourceMitigation']
3 ['ID', 'has_Category', 'NAME', 'PBNTYPE', 'has_Closest', 'has_RiskMitigation', 'has_RiskDescription', 'has_RiskGBN', 'has_RiskHealth', 'has_RiskISO_Impact', 'has_RiskISO_Purpose', 'has_RiskName', 'has_RiskOwner', 'has_RiskSource', 'has_RiskSubject', 'has_RiskTechnology', 'has_RiskType'] 

In [18]:
i = 0
for OneSheet in ALLDFs:
    title = OneSheet.iloc[0].PBNTYPE
    print(i,title,len(OneSheet))
    i += 1

0 Benef 11066
1 BenefReturn 1606
2 Article 481590
3 Risk 36782
4 RiskGBN 1590
5 RiskMitigation 34434
6 RiskHealth 14
7 RiskType 22
8 Stakeholder 11190
9 Technology 57843
10 RiskISO_Impact 24
11 RiskISO_Purpose 18
12 StakeholderGroup 3912
13 StakeholderSubgroup 1943
14 TechGroup 3900
15 TechSubgroup 1811
16 Enabler 420
17 Intervention 81
18 Blueprint 395
19 BP_Category 13
20 BP_Theme 50
21 BP_Intervention 80
22 Mitigation 29827
23 PBNCategory 16805
24 RiskISO 42


In [19]:
with pd.ExcelWriter("outputs/pbn_t3_5.xlsx") as writer:
    i = 0
    for OneSheet in ALLDFs:
        title = OneSheet.iloc[0].PBNTYPE
        print(i,title,len(OneSheet))
        i += 1
        OneSheet[:(1048576-1)].to_excel(writer, sheet_name=title, index=False, engine="xlsxwriter")
        print("... done.")


0 Benef 11066
... done.
1 BenefReturn 1606
... done.
2 Article 481590
... done.
3 Risk 36782
... done.
4 RiskGBN 1590
... done.
5 RiskMitigation 34434
... done.
6 RiskHealth 14
... done.
7 RiskType 22
... done.
8 Stakeholder 11190
... done.
9 Technology 57843
... done.
10 RiskISO_Impact 24
... done.
11 RiskISO_Purpose 18
... done.
12 StakeholderGroup 3912
... done.
13 StakeholderSubgroup 1943
... done.
14 TechGroup 3900
... done.
15 TechSubgroup 1811
... done.
16 Enabler 420
... done.
17 Intervention 81
... done.
18 Blueprint 395
... done.
19 BP_Category 13
... done.
20 BP_Theme 50
... done.
21 BP_Intervention 80
... done.
22 Mitigation 29827
... done.
23 PBNCategory 16805
... done.
24 RiskISO 42
... done.


# Creating  index

### Creating general indexes

In [20]:
IDX = ""
IDX += "# Overview\n\n"
for k in onto.classes():
    PATH =  "./docs/"
    #PATH = "7.pages/" 
    if len(k.instances()):
        MM = str(k).split(".")[-1]
        if (not MM == "RiskMitigation") and (not MM == "PBNThing"):
            IDX += "* _["+MM+"]("+LINK+MM+"/index.md)_ -- "+str(len(k.instances()))+ " instances.\n"
with open(PATH+"index.md","w",encoding="utf-8") as f:
        f.write(IDX)

print(IDX)

# Overview

* _[Qid](https://github.com/mm80843/T3.5/blob/main/docs/Qid/index.md)_ -- 3636 instances.
* _[Benef](https://github.com/mm80843/T3.5/blob/main/docs/Benef/index.md)_ -- 1415 instances.
* _[BenefReturn](https://github.com/mm80843/T3.5/blob/main/docs/BenefReturn/index.md)_ -- 1606 instances.
* _[Article](https://github.com/mm80843/T3.5/blob/main/docs/Article/index.md)_ -- 376 instances.
* _[Risk](https://github.com/mm80843/T3.5/blob/main/docs/Risk/index.md)_ -- 3736 instances.
* _[RiskGBN](https://github.com/mm80843/T3.5/blob/main/docs/RiskGBN/index.md)_ -- 5 instances.
* _[RiskHealth](https://github.com/mm80843/T3.5/blob/main/docs/RiskHealth/index.md)_ -- 14 instances.
* _[RiskType](https://github.com/mm80843/T3.5/blob/main/docs/RiskType/index.md)_ -- 22 instances.
* _[Stakeholder](https://github.com/mm80843/T3.5/blob/main/docs/Stakeholder/index.md)_ -- 3356 instances.
* _[Technology](https://github.com/mm80843/T3.5/blob/main/docs/Technology/index.md)_ -- 4968 instances.
* _[

### Creating classes indexes

In [21]:
LINK = "https://github.com/mm80843/T3.5/blob/main/docs/"
for k in onto.classes():
    K = str(k).split(".")[-1]
    if len(k.instances()):
    #print(K)
        PATH =  "./docs/"
        #PATH = "7.pages/" 
        if K not in ["PBNThing","Qid"]:
            IDX = "[Source](https://github.com/mm80843/T3.5/blob/main/docs/index.md)"
            IDX += "\n\n# "+str(K)+" -- "+str(len(k.instances()))+ " instances.\n\n"
            for i in k.instances(): 
                I = str(i).split(".")[-1]
                if K == "Qid":
                    if len(i.has_Qid_alias) > 0:

                        IDX += "* ["+i.has_Qid_alias[0]+"]("+LINK+"Qid/"+str(I)+".md)\n"
                    else:
                        IDX += "* ["+str(I)+"]\n"
                else:
                    II = I.split("PBN__")[1].split("_")[0]
                    A = i.label[0]
                    if K == "Article":
                        A = " ".join(list(set(A.split(" ")))).replace("nan","").strip()
                    IDX += "* ["+A+"]("+LINK+II+"/"+str(I)+".md)\n" 
            with open(PATH+K+"/index.md","w",encoding="utf-8") as f:
                f.write(IDX)

print(IDX)

[Source](https://github.com/mm80843/T3.5/blob/main/docs/index.md)

# RiskISO -- 42 instances.

* [Resilience](https://github.com/mm80843/T3.5/blob/main/docs/RiskISO/PBN__RiskISO_Impact_2.md)
* [Health and care](https://github.com/mm80843/T3.5/blob/main/docs/RiskISO/PBN__RiskISO_Impact_3.md)
* [Responsible resource use](https://github.com/mm80843/T3.5/blob/main/docs/RiskISO/PBN__RiskISO_Impact_4.md)
* [Well-being](https://github.com/mm80843/T3.5/blob/main/docs/RiskISO/PBN__RiskISO_Impact_1.md)
* [Preserv. and improve. of the environment](https://github.com/mm80843/T3.5/blob/main/docs/RiskISO/PBN__RiskISO_Impact_7.md)
* [Topic](https://github.com/mm80843/T3.5/blob/main/docs/RiskISO/PBN__RiskISO_Impact_12.md)
* [Economy and sustainable production and consumption](https://github.com/mm80843/T3.5/blob/main/docs/RiskISO/PBN__RiskISO_Impact_10.md)
* [Safety and security](https://github.com/mm80843/T3.5/blob/main/docs/RiskISO/PBN__RiskISO_Impact_0.md)
* [Preservation and improvement of the env

In [23]:
print("DONE")

DONE
